<a href="https://colab.research.google.com/github/rafaellopezgarcia/colab_notebooks/blob/main/MiniGPT_replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini GPT

In [ ]:
from pathlib import Path
from tabulate import tabulate

import torch

## Hyperparameters

In [ ]:
batch_size = 8
block_size = 8

## Data

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
data = ""
with open(Path('input.txt'), 'r') as tiny:
   data = tiny.read()

print(data[:500])

dataset_size = len(data)
training_set = data[:int(dataset_size*0.8)]
test_set = data[int(dataset_size*0.8):]

## Encoder-Decoder

In [ ]:
characters = set(data)
vocab_size = len(characters)
dataset_size = len(data)

stoi = {x: ix for ix, x in enumerate(characters)}
itos = {ix: x for ix, x in enumerate(characters)}

encode = lambda txt : [stoi[ch] for ch in txt]
decode = lambda nms : [itos[n] for n in nms]
example = decode(encode(data[:14]))
print(''.join(example))

#tokens = encode(data)
etrainining_set = torch.tensor(encode(training_set))
etest_set = torch.tensor(encode(test_set))

## Batches

In [ ]:
# Example of an already decoded block
x = training_set[0:batch_size]
y = training_set[1:batch_size+1]
table = []
for ix in range(len(x)):
  table.append([''.join(x[:ix+1]), ''.join(y[ix])])

print(tabulate(table, headers=['context','target']))

In [ ]:
torch.manual_seed(1337)


def get_batch(split='train'):
  data = etrainining_set if split=='train' else etest_set
  ixs = torch.randint(0, len(data) - block_size, (batch_size,))

  xb = torch.stack([data[ix:ix+block_size] for ix in ixs])
  yb = torch.stack([data[ix+1:ix+block_size+1] for ix in ixs])
  return xb, yb

print()
xb, yb = get_batch()
table = []
for ix in range(len(xb[0])):
  print(f"Context tensor: {xb[0][:ix+1].tolist()} | target: {yb[0][ix].item()}")

